In [140]:
import sklearn
import pickle
import numpy as np
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

In [50]:
stemmer = PorterStemmer()
stop_words = stopwords.words('english')
titles = ['biotechnology', 'business_economics', 'chemical_engineering', 'computer_science', 'electrical_engineering', 'geography', 'hospitality_&_tourism', 'law_school', 'medical_school', 'visual_design']

In [45]:
file = open("bernoulli_model_v1_c10_e230.pickle", "rb")
bernoulli = pickle.load(file)
file.close()
file = open("mnb_model_v1_c10_e230.pickle", "rb")
multinomial = pickle.load(file)
file.close()
file = open("logistic_model_v1_c10_e230.pickle", "rb")
logistic_regression = pickle.load(file)
file.close()

In [39]:
class VoteClassifier ():
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    def classify(self, prep_data):
        votes = [0] * len(self._classifiers[0]['clf'].classes_)
        for c in self._classifiers:
            v = c.predict_proba(prep_data)
            votes += v
        votes/=3
        return votes

In [159]:
from nltk.corpus import wordnet
def normalize_negation(text):
    words = word_tokenize(text)
    negation = ['not', "n't", "nt"]
    document = []
    i = 0
    while i < len(words):
        if any(string in words[i] for string in negation):
            antonym_found = False
            synset = wordnet.synsets(words[i+1])
            for s in synset:
                for synonym in s.lemmas():
                    for antonym in synonym.antonyms():
                        document.append(antonym.name())
                        antonym_found = True
                        break
                    if antonym_found == True:
                        break
                if antonym_found == True:
                    break
            i+=1
        else:
            document.append(words[i])
        i+=1
    return document

In [235]:
s = "i like drawing abstract painting to impress my girlfriend"

In [236]:
data = [s]
prep_data = []
# data = normalize_negation(data)

for d in data:
    d = [stemmer.stem(dt).lower() for dt in d.split(' ')]
    d = [dt for dt in d if dt not in stop_words]
    prep_data.append(' '.join(d))

In [166]:
prep_data

['want game develop']

In [167]:
classifier = VoteClassifier(bernoulli,multinomial, logistic_regression)

In [237]:
res = classifier.classify(prep_data)
print(res)
print(titles[np.argmax(res)])

[[0.06298904 0.0454247  0.0424167  0.05292984 0.0570761  0.20710952
  0.05010223 0.06710173 0.12930643 0.2855437 ]]
visual_design


In [82]:
titles

['biotechnology',
 'business_economics',
 'chemical_engineering',
 'computer_science',
 'electrical_engineering',
 'geography',
 'hospitality_&_tourism',
 'law_school',
 'medical_school',
 'visual_design']